In [1]:
import torch
import tensorflow as tf
import warnings
import os

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"

In [3]:
#  Define experimental specifics:
#  Check before run!
source_path = "E:\\research\\experiments\\128\\min_len\\bert-base_50_all\\"
output_path = "E:\\research\\experiments\\128\\min_len\\bert-base_50_all\\output"
#  512 for bert-base and 128 for bertweet
max_len = 128
#  for classification (specific authors + 1 dump for rest) minus blacklist-len
number_of_labels = 50
#  training epochs: lower when overfitting! (2 or 4, 30 in bertweet paper)
epochs = 8
#  random seed (original: 66)
seed_val = 66

In [4]:
device_name = tf.test.gpu_device_name()
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('=== Experiment' + source_path[source_path.rfind('bert'):]  + '===\n')
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
        cf.write('Used device: {}\n'.format(torch.cuda.get_device_name(0)))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
        # noinspection PyStringFormat
        cf.write('Used device: {:,}\n'.format("cpu"))

There are 1 GPU(s) available.
We will use the GPU: GRID A100-20C


In [5]:
import pandas as pd

train_df = pd.read_csv(source_path + "training_set.csv", sep='\t', encoding='utf-8')
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Number of comments: {:,}\n'.format(train_df.shape[0]))

train_df.sample(10)
sentences = train_df.Comment.values
labels = train_df.Author.values

Number of training sentences: 20,600



In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Used tokenizer: {}\n'.format("BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)"))
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))
for sent in sentences:
    input_ids = tokenizer.encode(str(sent), add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print('Max sentence length: ', max_len)
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Max tokens: {}\n'.format(max_len))
# Tokenize all the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
        str(sent),  # Sentence to encode.
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=max_len,  # Pad & truncate all sentences.
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt',  # Return pytorch tensors.
    )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
print('Original: ', sentences[1])
print('Token IDs:', input_ids[1])

 Original:  You could hear the host basically gasp once I said it hahaha.
Tokenized:  ['you', 'could', 'hear', 'the', 'host', 'basically', 'gasp', 'once', 'i', 'said', 'it', 'ha', '##ha', '##ha', '.']
Token IDs:  [2017, 2071, 2963, 1996, 3677, 10468, 12008, 2320, 1045, 2056, 2009, 5292, 3270, 3270, 1012]
Max sentence length:  159
Original:  This is nothing new for Trump.
Token IDs: tensor([ 101, 2023, 2003, 2498, 2047, 2005, 8398, 1012,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,

In [7]:
from torch.utils.data import TensorDataset, random_split
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)
# Create a 90-10 train-validation split.
# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Checking whether the distribution of target is consistent across both the sets
label_temp_list = []
for a, b, c in train_dataset:
    label_temp_list.append(c)
print('{:>5,} training samples'.format(train_size))
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Number of training comments: {}\n'.format(train_size))
label_temp_list = []
for a, b, c in val_dataset:
    label_temp_list.append(c)
print('{:>5,} validation samples'.format(val_size))
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Number of validation comments: {}\n'.format(val_size))

18,540 training samples
2,060 validation samples


In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Batch size for fine-tuning: {}\n'.format(batch_size))
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
    train_dataset,  # The training samples.
    sampler=RandomSampler(train_dataset),  # Select batches randomly
    batch_size=batch_size  # Trains with this batch size.
)
# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
    val_dataset,  # The validation samples.
    sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
    batch_size=batch_size  # Evaluate with this batch size.
)

In [9]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=number_of_labels, output_attentions=True, output_hidden_states=True)
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Operating model: {}\n'.format('BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=number_of_labels, output_attentions=True, output_hidden_states=True)'))
model.cuda()
# Get all of model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix'
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=5e-5,  # args.learning_rate - default is 5e-5, 1.e-5 in BERTweet paper
                              eps=1e-8  # args.adam_epsilon  - default is 1e-8.
                            )
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Optimizer: AdamW(model.parameters(), lr = 5e-5, eps = 1e-8)\n')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [10]:
from transformers import get_linear_schedule_with_warmup
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 2,I have already seen that the model starts overfitting beyond 2 epochs
# = 2  # 30 in BERTweet paper
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Epochs: {:,}\n'.format(epochs))
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps)

In [11]:
# Function to calculate the accuracy of our predictions vs labels
# noinspection PyShadowingNames
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [12]:
import time
import datetime
# noinspection PyShadowingNames
def format_time(elapsed):
    # Round to the nearest second.
    # noinspection PyRedundantParentheses
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [13]:
import random
import numpy as np
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
#seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []
# Measure the total training time for the whole run.
total_t0 = time.time()
# For each epoch...
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_train_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()
        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        loss = outputs.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            # Accumulate the validation loss.
        CUDA_LAUNCH_BLOCKING = 1
        loss = outputs.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 8 ========
Training...
  Batch    40  of    580.    Elapsed: 0:00:11.
  Batch    80  of    580.    Elapsed: 0:00:21.
  Batch   120  of    580.    Elapsed: 0:00:31.
  Batch   160  of    580.    Elapsed: 0:00:41.
  Batch   200  of    580.    Elapsed: 0:00:51.
  Batch   240  of    580.    Elapsed: 0:01:01.
  Batch   280  of    580.    Elapsed: 0:01:11.
  Batch   320  of    580.    Elapsed: 0:01:21.
  Batch   360  of    580.    Elapsed: 0:01:30.
  Batch   400  of    580.    Elapsed: 0:01:40.
  Batch   440  of    580.    Elapsed: 0:01:50.
  Batch   480  of    580.    Elapsed: 0:02:00.
  Batch   520  of    580.    Elapsed: 0:02:10.
  Batch   560  of    580.    Elapsed: 0:02:20.

  Average training loss: 1.76
  Training epoch took: 0:02:25

Running Validation...
  Accuracy: 0.76
  Validation Loss: 0.93
  Validation took: 0:00:05

======== Epoch 2 / 8 ========
Training...
  Batch    40  of    580.    Elapsed: 0:00:10.
  Batch    80  of    580.    Elapsed: 0:00:20.
  Batch  

In [14]:
import pandas as pd
# Display floats with two decimal places.
pd.set_option('display.precision', 2)
# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)
# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write("Fine-tuning stats:\n" + df_stats.to_string() + "\n")
# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.76e+00,0.93,0.76,0:02:25,0:00:05
2,6.45e-01,0.69,0.82,0:02:23,0:00:05
3,2.93e-01,0.70,0.82,0:02:23,0:00:05
4,1.35e-01,0.75,0.82,0:02:23,0:00:05
5,6.02e-02,0.78,0.83,0:02:24,0:00:05
6,2.40e-02,0.80,0.83,0:02:26,0:00:06
7,1.01e-02,0.79,0.84,0:02:27,0:00:05
8,4.98e-03,0.81,0.83,0:02:27,0:00:05


In [15]:
import pandas as pd
from importlib import reload
from trainingdata_preparation import tdp as tdp
reload(tdp)
# Load the dataset into a pandas dataframe.
df = pd.read_csv(source_path + "testing_set.csv", sep='\t', encoding="utf-8")
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))
# Create sentence and label lists
sentences = df.Comment.values
labels = []
for i in range(0, number_of_labels):
    labels.append(i)
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write('Classification labels: {}\n'.format(labels))
# Tokenize all the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []
# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
        str(sent),  # Sentence to encode.
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=max_len,  # Pad & truncate all sentences.
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt',  # Return pytorch tensors.
    )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
# Set the batch size.  
batch_size = 32
# Create the DataLoader.
# prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions, true_labels = [], []
# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    # b_input_ids, b_input_mask, b_labels = batch
    b_input_ids, b_input_mask = batch
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    # label_ids = b_labels.to('cpu').numpy()
    # Store predictions and true labels
    predictions.append(logits)
    # true_labels.append(label_ids)
print('    DONE.')
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)
# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# adding to the main dataframe
df['target'] = flat_predictions
# downloading file to local
df.to_csv(source_path + 'prediction.tsv', index=False, sep="\t")
with open(source_path + "comparative_data.txt", 'a', encoding='utf-8') as cf:
    cf.write(tdp.evaluation(source_path + 'testing_val_set.csv', source_path + "prediction.tsv", number_of_labels))

Number of test sentences: 5,150

Predicting labels for 5,150 test sentences...
    DONE.


In [16]:
import os
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
# Create output directory if needed
if not os.path.exists(output_path):
    os.makedirs(output_path)
print("Saving model to %s" % output_path)
# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

Saving model to E:\research\experiments\128\min_len\bert-base_50_all\output


('E:\\research\\experiments\\128\\min_len\\bert-base_50_all\\output\\tokenizer_config.json',
 'E:\\research\\experiments\\128\\min_len\\bert-base_50_all\\output\\special_tokens_map.json',
 'E:\\research\\experiments\\128\\min_len\\bert-base_50_all\\output\\vocab.txt',
 'E:\\research\\experiments\\128\\min_len\\bert-base_50_all\\output\\added_tokens.json')